In [19]:
from SQP1Dataset import initialize_datasets, SQP1Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Optimizer, AdamW
from tqdm.notebook import tqdm

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

datasets = initialize_datasets('../data/train.json', '../data/dev.json', tokenizer)

train_dataloader = DataLoader(datasets['train'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=SQP1Dataset.collate_fn)

validation_dataloader = DataLoader(datasets['dev'],
                                   batch_size=64,
                                   shuffle=False,
                                   collate_fn=SQP1Dataset.collate_fn)

# batch = next(iter(validation_dataloader))
# print(batch.keys())

print(f"{len(datasets['dev'])}")

229


In [27]:
#### THIS WORKS WITHOUT ERRORS
input_question = "Are more people today related to Genghis Khan than Julius Caesar?"
decompositions = [
            "How many kids did Julius Caesar have?",
            "How many kids did Genghis Khan have?",
            "Is #2 greater than #1?"
        ]

inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
outputs = tokenizer("<SEP>".join(decompositions), return_tensors="pt", padding=True, truncation=True)
print(inputs)
print(outputs)
# Fine-tuning
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for epoch in range(2):
    optimizer.zero_grad()
    loss = model(**inputs, labels=outputs['input_ids'], decoder_input_ids=outputs['input_ids']).loss
    loss.backward()
    optimizer.step()

{'input_ids': tensor([[ 1521,    72,   151,   469,  1341,    12,  5945,  5649,     7, 14420,
           145,  9983,   302, 26218,    58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
{'input_ids': tensor([[  571,   186,  1082,   410,  9983,   302, 26218,    43,    58,     2,
           134,  8569,  3155,  7825,   186,  1082,   410,  5945,  5649,     7,
         14420,    43,    58,     2,   134,  8569,  3155,   196,     7, 15493,
          2123,   145,  7172,    58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [2]:
def train_one_epoch(model: nn.Module, dataloader: DataLoader, optimizer: Optimizer, epoch: int):

    model.train()

    with tqdm(dataloader, desc=f"Train Ep {epoch}", total=len(dataloader)) as tq:
        for batch in tq:
            text_encoding = batch['input_ids']
            label_encoding = batch['target_ids']

            # TODO: Compute loss by running model with text_encoding and label_encoding.
            loss = model(**text_encoding, labels=label_encoding).loss

            # TODO: compute gradients and update parameters using optimizer.
            # Hint: you need three lines of code here!
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #tq.set_postfix({"loss": loss.detach().item()}) # for printing better-looking progress bar

In [3]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
train_one_epoch(model, train_dataloader, optimizer, 1)

Train Ep 1:   0%|          | 0/33 [00:00<?, ?it/s]

TypeError: T5Model(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0